In [ ]:
# 回到 FINRL/
%cd ..

In [ ]:
from AI_Trading.src import config
from AI_Trading.src.env_portfolio_allocation import *
from AI_Trading.src.evaluate import *
from AI_Trading.src import model_config
from AI_Trading.src.preprocess import *
from AI_Trading.src.testPortfolio import *
from AI_Trading.src.train import *
from AI_Trading.src.augmentation import *
from stable_baselines3 import A2C, DDPG, PPO, SAC, TD3
import pandas as pd
from tqdm import tqdm
from warnings import simplefilter
simplefilter(action='ignore')
import matplotlib.pyplot as plt
matplotlib.use('Agg')
%matplotlib inline

In [ ]:
#evaluate setting
acitonFile_path = './AI_Trading/results/base/'
exp = 'base'
save_path = os.path.join(config.EVALUATE_RESULT_PATH, exp)
if not os.path.isdir(save_path):
    try:
        os.mkdir(save_path)
    except Exception:
        print(f'no folder {save_path}')
        pass

In [ ]:
df_rank_allYear = pd.DataFrame()
df_annualReturn_allYear = pd.DataFrame()
df_mdd_allYear = pd.DataFrame()
df_sharpe_allYear = pd.DataFrame()
df_sortino_allYear = pd.DataFrame()

# evaluate
for i in tqdm(range(len(config.TEST_START_DATE))):
    train,trade = preprocess(config.TRAIN_START_DATE[i], config.TRAIN_END_DATE[i], config.TEST_START_DATE[i], config.TEST_END_DATE[i])

    # load actions file
    df_actions = pd.read_csv(f'{acitonFile_path}df_action_{exp}_{i}.csv')
    df_return, df_portfolio_value = computeReturns(df_actions,trade)
    returns, stats = getStats(df_return)

    # Equal weight
    equalWeight_actions = getEqualWeightActions(trade)
    df_equalWeight_return, df_portfolio_value = computeReturns(equalWeight_actions,trade)
    equalWeight_returns, equalWeight_stats = getStats(df_equalWeight_return)

    # Min-Variance
    minVariance, minVariance_stats = getMinVariance(trade)

    # All In Stock
    all_stock_actions= getTicActions(trade, 'VTI')
    df_stock_return, df_stock_value = computeReturns(all_stock_actions,trade)
    stock_returns, stock_stats = getStats(df_stock_return)

    # All In Debt
    all_debt_actions= getTicActions(trade, 'TLT')
    df_debt_return, df_debt_value = computeReturns(all_debt_actions,trade)
    debt_returns, debt_stats = getStats(df_debt_return)

    # All In Reit
    all_reit_actions= getTicActions(trade, 'VNQ')
    df_reit_return, df_reit_value = computeReturns(all_reit_actions,trade)
    reit_returns, reit_stats = getStats(df_reit_return)
    
    df_stats = pd.concat({'Ours': stats,
                          'equalWeight': equalWeight_stats,
                          'minVariance':minVariance_stats,
                          'stock': stock_stats,
                          'debt': debt_stats,
                          'reit': reit_stats},axis=1)

    # get each year stats
    df_annualReturn_allYear = stats_allYear(i, 'Annual return', df_stats, df_annualReturn_allYear)
    df_mdd_allYear = stats_allYear(i, 'Max drawdown', df_stats, df_mdd_allYear)
    df_sharpe_allYear = stats_allYear(i, 'Sharpe ratio', df_stats, df_sharpe_allYear)
    df_sortino_allYear = stats_allYear(i, 'Sortino ratio', df_stats, df_sortino_allYear)           
    
    # get each rank
    df_rank_allYear = rankCaculate(i, 'Sharpe ratio', df_stats, df_rank_allYear)
    df_rank_allYear_s = rankCaculate(i, 'Sortino ratio', df_stats, df_rank_allYear)

In [ ]:
df_annualReturn_allYear_avg = average_allYear(df_annualReturn_allYear)
df_annualReturn_allYear_avg.to_csv(f'{config.EVALUATE_RESULT_PATH}{exp}/df_annualReturn.csv')
df_annualReturn_allYear_avg

In [ ]:
df_mdd_allYear_avg = average_allYear(df_mdd_allYear)
df_mdd_allYear_avg.to_csv(f'{config.EVALUATE_RESULT_PATH}{exp}/df_mdd.csv')
df_mdd_allYear_avg

In [ ]:
df_sharpe_allYear_avg = average_allYear(df_sharpe_allYear)
df_sharpe_allYear_avg.to_csv(f'{config.EVALUATE_RESULT_PATH}{exp}/df_sharpe.csv')
df_sharpe_allYear_avg

In [ ]:
df_sortino_allYear_avg = average_allYear(df_sortino_allYear)
df_sortino_allYear_avg.to_csv(f'{config.EVALUATE_RESULT_PATH}{exp}/df_sortino.csv')
df_sortino_allYear_avg

In [ ]:
df_rank_allYear_avg = average_allYear(df_rank_allYear)
df_rank_allYear_avg